# Part II — Core Django  
## 14. Static Files and Media Files (Correct Setup, Real Uploads, Dev vs Production)

This chapter formalizes two things that *every* real Django app must get right:

1. **Static files**: assets shipped with your code (CSS/JS/images).
2. **Media files**: user-uploaded files (avatars, documents, product photos).

Beginners often mix these up. In production, treating them incorrectly leads to:
- broken CSS/JS after deploy
- missing user uploads
- security vulnerabilities (serving private files publicly)
- performance issues (Django serving files instead of a proper web server/CDN)

We’ll implement:
- a clean static file structure
- a real file upload stored on disk in dev
- correct settings and URL wiring
- security validation patterns for uploads
- templates that display uploaded images/files
- tests for upload behavior

---

## 14.0 Learning Outcomes

By the end you will be able to:

1. Explain static vs media with production implications.
2. Configure:
   - `STATIC_URL`, `STATICFILES_DIRS`, `STATIC_ROOT`
   - `MEDIA_URL`, `MEDIA_ROOT`
3. Serve media in development safely.
4. Add a model with `ImageField`/`FileField` and upload via ModelForm.
5. Validate uploaded files (size/type) and avoid common security mistakes.
6. Understand `collectstatic` and why production serving is different.
7. Write tests using Django’s test client to upload files.

---

## 14.1 Static vs Media: Definitions You Must Never Forget

### Static files (code-owned)
- Created by developers
- Versioned with git
- Deployed with the application
- Examples:
  - `site.css`
  - `app.js`
  - logo images used by UI

### Media files (user-owned)
- Uploaded by users or generated dynamically
- Not part of your code repo
- Must persist across deploys
- Requires backup and storage strategy
- Examples:
  - profile photo uploads
  - article cover image uploads
  - invoices PDFs

**Industry reality**
- Static is usually served via CDN or Nginx after being “collected.”
- Media is stored on persistent storage (disk volume, S3, etc.) and served separately.

---

## 14.2 Static Files Configuration (Correct, Professional Setup)

Open `config/settings.py`.

### 14.2.1 Core static settings
You typically have:

```python
STATIC_URL = "static/"
```

This means: “static files are accessible under the URL prefix `/static/`.”

### 14.2.2 Where static files live during development: `STATICFILES_DIRS`
You already created:

```text
static/css/site.css
```

To tell Django to look in `BASE_DIR/static/`, set:

```python
STATICFILES_DIRS = [
    BASE_DIR / "static",
]
```

Explanation:
- This is a list of directories where you keep static assets during development.
- Django will search these directories to serve static assets.

### 14.2.3 Where collected static files go for production: `STATIC_ROOT`
Add:

```python
STATIC_ROOT = BASE_DIR / "staticfiles"
```

Explanation:
- In production, you run `collectstatic`.
- Django copies all static assets from apps + directories into `STATIC_ROOT`.
- Then your web server (Nginx/CDN) serves from `STATIC_ROOT`.

**Key point:** `STATIC_ROOT` is usually *not* the same as your dev `static/` folder.

A common professional layout:
- `static/` → source assets you edit
- `staticfiles/` → generated collection output

You generally do not commit `staticfiles/` to git.

---

## 14.3 How Django Finds Static Files (The Staticfiles Finders)

When `django.contrib.staticfiles` is installed (it is by default), Django uses
“finders” to locate static files in:

1. `STATICFILES_DIRS`
2. each installed app’s `static/` directory:
   - `myapp/static/myapp/...`

Example app static layout:

```text
articles/
  static/
    articles/
      articles.css
```

This mirrors the templates namespacing pattern. Namespacing avoids collisions.

---

## 14.4 Serving Static Files in Development

With `DEBUG=True`, Django serves static files automatically when you run:

```bash
python manage.py runserver
```

This is only for development. In production, Django should not serve static files.

### Verify static
Visit:
- `http://127.0.0.1:8000/static/css/site.css`

If you see CSS content, your static setup works.

---

## 14.5 `collectstatic` (What It Does and Why It Exists)

Run:

```bash
python manage.py collectstatic
```

Django will copy static assets into `STATIC_ROOT`.

### 14.5.1 Why production needs `collectstatic`
In production, you often have:
- static files in multiple apps
- pipeline tools (bundlers, hashed filenames)
- a web server that wants one directory to serve static from

`collectstatic`:
- collects everything into a single directory
- optionally applies storage transformations (like hashed names) depending on storage backend

### 14.5.2 What you should not do
Do not rely on Django’s dev static serving in production. It’s slower and not designed
for high traffic.

---

## 14.6 Media Files Setup (Where Uploads Are Stored)

Add to `config/settings.py`:

```python
MEDIA_URL = "/media/"
MEDIA_ROOT = BASE_DIR / "media"
```

Explanation:
- `MEDIA_ROOT` is the filesystem directory where uploaded files are stored.
- `MEDIA_URL` is the URL prefix used to access them (in dev) or served by web server.

### 14.6.1 Why media is not automatic like static
Serving user uploads is a production decision:
- public vs private
- storage backend (local disk vs S3)
- access control
- caching and CDN behavior

In dev, we serve them via Django for convenience, but production typically uses
Nginx/S3.

---

## 14.7 Serve Media in Development (Correct URL Wiring)

In `config/urls.py`, add development-only media serving.

```python
from django.contrib import admin
from django.conf import settings
from django.conf.urls.static import static
from django.urls import include, path

urlpatterns = [
    path("admin/", admin.site.urls),
    path("", include("pages.urls")),
    path("articles/", include("articles.urls")),
    path("accounts/", include("django.contrib.auth.urls")),
]

if settings.DEBUG:
    urlpatterns += static(settings.MEDIA_URL, document_root=settings.MEDIA_ROOT)
```

### Explanation
- `static(...)` here is a helper to serve files during development.
- Wrapping it in `if settings.DEBUG:` ensures it doesn’t run in production.
- In production, you must serve media using a proper solution (later chapters).

---

## 14.8 Real Feature: Add a Cover Image Upload to Article

We’ll add:

- `cover_image = ImageField(...)` to `Article`
- update form and templates
- implement upload validation
- show the image in article detail page

### 14.8.1 Install Pillow (required for ImageField)
`ImageField` requires Pillow.

Install:

```bash
python -m pip install pillow
python -m pip freeze > requirements.txt
```

### 14.8.2 Update the model
Edit `articles/models.py`:

```python
class Article(models.Model):
    ...
    cover_image = models.ImageField(
        upload_to="article_covers/",
        null=True,
        blank=True,
    )
    ...
```

#### Explain `upload_to`
- It sets the subdirectory under `MEDIA_ROOT` where files are stored.
- Example: `MEDIA_ROOT/article_covers/myimage.jpg`

Run migrations:

```bash
python manage.py makemigrations
python manage.py migrate
```

---

## 14.9 Update the Form to Handle File Uploads (Correctly)

### 14.9.1 Add field and validation in `ArticleForm`
Edit `articles/forms.py` and ensure `cover_image` is included.

```python
class ArticleForm(forms.ModelForm):
    class Meta:
        model = Article
        fields = [
            "title",
            "slug",
            "body",
            "status",
            "published_at",
            "cover_image",
            "tags",
        ]
```

### 14.9.2 Validate file size/type (basic but important)
Add to `ArticleForm`:

```python
from django.core.exceptions import ValidationError

def clean_cover_image(self):
    image = self.cleaned_data.get("cover_image")
    if not image:
        return image

    max_bytes = 2 * 1024 * 1024  # 2MB
    if image.size > max_bytes:
        raise ValidationError("Cover image too large (max 2 MB).")

    # Content-type checks are not perfect, but useful as a first layer.
    allowed = {"image/jpeg", "image/png", "image/webp"}
    content_type = getattr(image, "content_type", None)
    if content_type and content_type not in allowed:
        raise ValidationError("Unsupported image type. Use JPG/PNG/WebP.")

    return image
```

#### Security explanation (important)
- Client-provided `content_type` can be faked; it’s not a full guarantee.
- Pillow will parse image file; corrupted files may raise errors during processing.
- For higher security, you can verify file signatures (advanced) or re-encode images.
- Still, size/type checks prevent common misuse and DoS risk.

---

## 14.10 Update Create/Edit Views to Pass `request.FILES`

This is the most common upload bug.

In `articles/views.py` update:

### Create:
```python
form = ArticleForm(request.POST, request.FILES)
```

### Edit:
```python
form = ArticleForm(request.POST, request.FILES, instance=article)
```

If you forget `request.FILES`:
- the form will treat the file field as empty
- upload will silently not save

---

## 14.11 Update Form Template to Use Multipart Encoding

In `articles/templates/articles/form.html`, update `<form>` tag:

```django
<form method="post" enctype="multipart/form-data">
```

Without this:
- browser does not send file bytes properly.

### Add cover image field rendering
Add a field section in your form template:

```django
<div class="field">
  <label for="{{ form.cover_image.id_for_label }}">Cover image</label>
  {{ form.cover_image }}
  {% for err in form.cover_image.errors %}<div class="error">{{ err }}</div>{% endfor %}
  {% if mode == "edit" and article.cover_image %}
    <p>Current: <a href="{{ article.cover_image.url }}">{{ article.cover_image.name }}</a></p>
    <img src="{{ article.cover_image.url }}" alt="Cover image" style="max-width: 300px;" />
  {% endif %}
</div>
```

#### Explain `.url` and `.name`
For a FileField/ImageField, Django provides:
- `.name`: path relative to media storage (`article_covers/xyz.jpg`)
- `.url`: URL for accessing it (works in dev with media serving configured)

---

## 14.12 Display Cover Image on Article Detail Page

Edit `articles/templates/articles/detail.html`:

```django
{% if article.cover_image %}
  <figure>
    <img
      src="{{ article.cover_image.url }}"
      alt="Cover for {{ article.title }}"
      style="max-width: 600px;"
    />
  </figure>
{% endif %}
```

Now:
- upload image in create/edit
- view the detail page
- image is served from `/media/...` in development

---

## 14.13 Add Media Directory to `.gitignore`

Media is user-generated; don’t commit it.

In `.gitignore` ensure you have:

```gitignore
media/
```

---

## 14.14 Testing File Uploads (Correct Test Pattern)

Django provides utilities to simulate uploaded files.

Create `articles/tests_uploads.py`:

```python
from django.contrib.auth import get_user_model
from django.core.files.uploadedfile import SimpleUploadedFile
from django.test import TestCase
from django.urls import reverse

from articles.models import Article


class ArticleUploadTests(TestCase):
    def setUp(self):
        User = get_user_model()
        self.user = User.objects.create_user(username="u1", password="pass12345")
        self.client.login(username="u1", password="pass12345")

    def test_upload_cover_image_creates_article(self):
        # Minimal fake "image" bytes. For strict image validation you may need
        # real image bytes; this test focuses on request wiring.
        file = SimpleUploadedFile(
            "cover.png",
            b"\x89PNG\r\n\x1a\n" + b"fakepngdata",
            content_type="image/png",
        )

        url = reverse("articles:create")
        response = self.client.post(
            url,
            {
                "title": "With Image",
                "slug": "with-image",
                "body": "Body content long enough.",
                "status": "draft",
                "published_at": "",
                "cover_image": file,
            },
        )

        self.assertEqual(response.status_code, 302)
        a = Article.objects.get(slug="with-image")
        self.assertTrue(bool(a.cover_image))
        self.assertIn("article_covers/", a.cover_image.name)
```

### Important test note
If your validation relies on Pillow truly decoding the image, fake bytes may fail.
In that case:
- include a small real PNG fixture file in tests
- or generate a real image in memory (advanced)

For workbook purposes, the key is:
- `request.FILES` passed
- form uses multipart encoding
- model saves file name/path

---

## 14.15 Production Guidance (What Changes Outside Dev)

### 14.15.1 Static in production
Typical production setup:
- run `collectstatic` during build/deploy
- serve `STATIC_ROOT` via:
  - Nginx
  - CDN (CloudFront, etc.)
  - platform static hosting

Django should not serve static in production.

### 14.15.2 Media in production
Common patterns:
- Local disk volume on the server (simplest, but you need persistent storage and
  backups; tricky with multiple servers)
- Object storage (S3-compatible) + CDN (most scalable)

Also:
- you may need private media (auth-protected)
- for private media, you do not serve directly by public URL; you use signed URLs or
  authenticated views.

We will cover these in deployment/security chapters.

---

## 14.16 Security Checklist for Uploads (Must-Know)

1. Limit file size (you did).
2. Validate type (content-type, extension, and/or signature).
3. Store uploads outside source code directories.
4. Do not allow users to upload executable files to publicly served directories
   without strict handling.
5. Never trust filename; Django’s storage will handle naming, but be mindful if you
   do custom naming.
6. For images:
   - consider re-encoding to a safe format
   - strip metadata (EXIF) if privacy matters (advanced)
7. Consider rate limiting and quotas at scale (advanced).

---

## 14.17 Exercises (Do These Before Proceeding)

1. Add a `Attachment` model (FileField) related to Article:
   - one Article can have many attachments (ForeignKey)
   - create an upload form for attachments
   - list attachments on article detail page
   - explain why this is media (not static)

2. Add file size limit for attachments and test it returns form errors.

3. Run `collectstatic` and confirm `staticfiles/` is created and contains your CSS.

4. Explain, in your own words:
   - why static can be cached aggressively but media often cannot
   - why media must be backed up differently than code

---

## 14.18 Chapter Summary

- Static files are code-owned assets served efficiently via collectstatic + web server/CDN.
- Media files are user uploads stored in MEDIA_ROOT and served separately.
- Uploads require:
  - multipart form encoding
  - passing `request.FILES`
  - model FileField/ImageField
- Media serving in dev uses `static(settings.MEDIA_URL, ...)` behind `DEBUG=True`.
- Production handling of static/media is a deployment architecture decision.
- Upload validation is essential for security and stability.

---

Next chapter: **Part II — Chapter 15: Email and Notifications**  
We’ll send emails for password reset/verification, build template-based emails, use
different email backends for dev vs production, and implement notification patterns
that scale (including background tasks preview).